<a href="https://colab.research.google.com/github/laiba-bajwa/ModelForge/blob/main/binary_classifier_accuracy_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk import pos_tag

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('averaged_preceptron_tagger_eng')
nltk.download('words')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Error loading averaged_preceptron_tagger_eng: Package
[nltk_data]     'averaged_preceptron_tagger_eng' not found in index
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
#file upload ho gi h
from google.colab import files
uploaded=files.upload()

Saving en_train.csv to en_train.csv


In [ ]:
#file read ho gi hai
import pandas as pd
df_test=pd.read_csv('en_train.csv')
print(df_test.head(7))
# Print the first few values of the 'binary' column before cleaning
#print(df_test[['text', 'binary']].head(20))

                                                text    binary  \
0  #USER# #USER# #USER# #USER# You expect a man t...  Not Hope   
1  #USER# #USER# #USER# #USER# Tinubu is actually...  Not Hope   
2  it'd be nice if missguided actually had stock ...      Hope   
3  #USER# Anyway love u bubbly i know i can count...      Hope   
4  “you have a lot of people rooting for you whet...  Not Hope   
5  Hoping to get on the brown a lot more after I ...  Not Hope   
6  #USER# no exactly :/ and like i’ve seen (non g...  Not Hope   

         multiclass  
0          Not Hope  
1          Not Hope  
2           Sarcasm  
3  Generalized Hope  
4          Not Hope  
5          Not Hope  
6          Not Hope  


In [ ]:
print(df_test.columns)
print(df_test.shape)
stemmer=PorterStemmer() #object
lemmatizer=WordNetLemmatizer()#object



def preprocess(tweet):
  if not isinstance(tweet,str):
    return" "

  tweet=tweet.lower()
  nonpunc=''.join([c for c in tweet if c not in string.punctuation])
  #print("Punctuation:", nonpunc)
  Stop_wprds=stopwords.words('english')
   #tokenize into words
  tokenize=word_tokenize(nonpunc)


  cleaned=[word for word in tokenize if word not in Stop_wprds]
  #print("Stopwords:", cleaned)

  words=[stemmer.stem(word)for word in cleaned]

  lem=[lemmatizer.lemmatize(text)for text in words]

  ct=' '.join(lem)
  return ct

df_test['clean']=df_test['text'].apply(preprocess)
print(df_test['clean'].head(9))

Index(['text', 'binary', 'multiclass'], dtype='object')
(5233, 3)
0    user user user user expect man liter refer god...
1    user user user user tinubu actual bonu lol hop...
2      itd nice missguid actual stock once������������
3     user anyway love u bubbl know count fairi tail😌🤚
4    “ lot peopl root whether believ ’ want let ” l...
5            hope get brown lot move part citi lot bar
6    user exactli like ’ seen non gvfx peopl say to...
7    user lost call cultur backward never progress ...
8    user user user user user user user user hi😊 ’ ...
Name: clean, dtype: object


In [ ]:
#df_test['binary']=df_test['binary'].map({'Hope':1,'Not Hope':0})
#print(df_test[['text', 'binary']].head(10))


# Define the binary mapping (e.g., 'negative' -> 0, 'positive' -> 1)
binary_mapping = {'Hope': 1, 'Not Hope': 0}

# Filter the dataset to keep only the rows where the sentiment is 'negative' or 'positive'
df_test = df_test[df_test['binary'].isin(binary_mapping.keys())].copy()

# Map the sentiment labels to numeric values (negative -> 0, positive -> 1)
df_test['binary'] = df_test['binary'].map(binary_mapping)

# Check the result
print(df_test[['text', 'binary']].head(10))

                                                text  binary
0  #USER# #USER# #USER# #USER# You expect a man t...       0
1  #USER# #USER# #USER# #USER# Tinubu is actually...       0
2  it'd be nice if missguided actually had stock ...       1
3  #USER# Anyway love u bubbly i know i can count...       1
4  “you have a lot of people rooting for you whet...       0
5  Hoping to get on the brown a lot more after I ...       0
6  #USER# no exactly :/ and like i’ve seen (non g...       0
7  #USER# You have lost because your so called cu...       0
8  #USER# #USER# #USER# #USER# #USER# #USER# #USE...       1
9  #USER# #USER# My dear Rwandans still remember ...       0


In [ ]:
print(df_test['binary'].dtype)

int64


In [ ]:
train_vocab=set(''.join(df_test['clean']).split())
test_vocab=set(''.join(df_test['clean']).split())

print("=== Vocabulary statistics ===")
print(f"Train unique words : {len(train_vocab):,}")
print(f"Test  unique words : {len(test_vocab):,}")

=== Vocabulary statistics ===
Train unique words : 16,950
Test  unique words : 16,950


In [ ]:
# Sorting the words alphabetically
"""
sorted_words = sorted(unique_words)
print("Sorted Words: ", sorted_words)
print("Number of Sorted Words: ", len(sorted_words))
"""

print("Sample train words :", sorted(list(train_vocab))[:10], "...\n")
print("Sample test  words :", sorted(list(test_vocab))[:10], "...\n")

Sample train words : ['0', '00', '0000', '000268958', '005', '007', '00700', '01', '010', '015'] ...

Sample test  words : ['0', '00', '0000', '000268958', '005', '007', '00700', '01', '010', '015'] ...



In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into 80% training and 20% testing
X = df_test['clean']  # Feature column
y = df_test['binary']  # Label column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

# Print the shapes of the split data
print(f"Training Data: {X_train.shape}, Testing Data: {X_test.shape}")


Training Data: (4186,), Testing Data: (1047,)


In [ ]:
# Machine Learning Model Training and Testing

#CountVectorizer: This method represents text based on the frequency of words.
#TF-IDF (Term Frequency-Inverse Document Frequency): This method weighs words based on their importance in a document relative to the entire corpus.

from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit the model on training data and transform both train and test data
X_train_cv = vectorizer.fit_transform(X_train)
X_test_cv = vectorizer.transform(X_test)

print(f"Training Data Shape after Vectorization: {X_train_cv.shape}")
print(f"Testing Data Shape after Vectorization: {X_test_cv.shape}")

print("\n vextorizer size of vocab : ", len(vectorizer.get_feature_names_out()))
print("Sample features:", vectorizer.get_feature_names_out()[:20])


Training Data Shape after Vectorization: (4186, 11632)
Testing Data Shape after Vectorization: (1047, 11632)
/n vextorizer size of vocab :  11632
Sample features: ['00' '0000' '000268958' '005' '007' '00700' '01' '010' '015' '02' '020'
 '025' '03' '04' '05' '06' '0620' '0625' '06272022' '07']


In [ ]:
from sklearn.naive_bayes import MultinomialNB
import time

start_time = time.time()
# Initialize the Multinomial Naive Bayes classifier
nb_model = MultinomialNB()

# Train the model on the training data
nb_model.fit(X_train_cv, y_train)

end_time = time.time()

print("\nTraining Time: {:.3f} seconds".format(end_time - start_time))
# Predict on the test data
y_pred = nb_model.predict(X_test_cv)

# Print predictions
print(f"Predictions on Test Data: {y_pred}")



Training Time: 0.004 seconds
Predictions on Test Data: [1 0 1 ... 1 1 0]


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print Confusion Matrix
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

# Print Classification Report
print("Classification Report: ")
print(classification_report(y_test, y_pred))


Accuracy: 75.84%
Confusion Matrix: 
[[442 120]
 [133 352]]
Classification Report: 
              precision    recall  f1-score   support

           0       0.77      0.79      0.78       562
           1       0.75      0.73      0.74       485

    accuracy                           0.76      1047
   macro avg       0.76      0.76      0.76      1047
weighted avg       0.76      0.76      0.76      1047



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit the model on training data and transform both train and test data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Print the shapes of the transformed data
print(f"Training Data Shape after TF-IDF: {X_train_tfidf.shape}")
print(f"Testing Data Shape after TF-IDF: {X_test_tfidf.shape}")


Training Data Shape after TF-IDF: (4186, 11632)
Testing Data Shape after TF-IDF: (1047, 11632)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_features = 20000,
    ngram_range = (1,2),
    min_df = 2,
    max_df = 0.95
)

In [ ]:
x_train_tfidf = vectorizer.fit_transform(X_train)
x_test_tfidf= vectorizer.transform(X_train)
print("\nTF-IDF Vectorizer vocabulary size:", len(vectorizer.get_feature_names_out()))
print("Sample features:", vectorizer.get_feature_names_out()[:5])


TF-IDF Vectorizer vocabulary size: 9155
Sample features: ['00' '007' '01' '010' '03']


In [ ]:
print(X_train.shape)       # original text
print(x_train_tfidf.shape) # after TF-IDF vectorization


(4186,)
(4186, 9155)
